# Phase 2B Walkthrough: Delta Lake Support

**Version:** v1.2.0-alpha.2-phase2b  
**Date:** November 9, 2025  
**Status:** ✅ Complete

---

## What You'll Learn

This walkthrough demonstrates:
1. ✅ Delta Lake read/write with PandasEngine
2. ✅ Delta Lake read/write with SparkEngine
3. ✅ Time travel (read specific versions)
4. ✅ VACUUM operations (clean old files)
5. ✅ History tracking
6. ✅ Restore to previous versions
7. ✅ Partitioning with warnings
8. ✅ Delta + ADLS integration

---

## Prerequisites

```bash
# Install ODIBI with Delta support
pip install -e ".[pandas,azure]"

# For Spark engine (optional)
pip install -e ".[spark,azure]"
```

## Part 1: Setup and Sample Data

In [ ]:
import pandas as pd
import sys
from pathlib import Path

# Add odibi to path
sys.path.insert(0, str(Path.cwd().parent))

from odibi.engine.pandas_engine import PandasEngine
from odibi.connections.local import LocalConnection

print("✅ ODIBI imported successfully")
print("\nPhase 2B: Delta Lake Support")
print("="*50)

In [ ]:
# Create sample data
sample_sales = pd.DataFrame({
    "order_id": [1, 2, 3, 4, 5],
    "customer": ["Alice", "Bob", "Charlie", "David", "Eve"],
    "product": ["Laptop", "Phone", "Tablet", "Laptop", "Phone"],
    "amount": [1200, 800, 400, 1200, 750],
    "year": [2024, 2024, 2024, 2024, 2024],
    "month": [1, 1, 2, 2, 3]
})

print("Sample Sales Data:")
print(sample_sales)
print(f"\nTotal: ${sample_sales['amount'].sum():,}")

In [ ]:
# Setup local connection and engine
engine = PandasEngine()
local_conn = LocalConnection(base_path="./delta_test_data")

print("✅ PandasEngine initialized")
print(f"✅ Local connection: {local_conn.base_path}")

## Part 2: Delta Lake Basics - Write and Read

In [ ]:
# Write Delta table (version 0)
delta_path = "sales.delta"

engine.write(
    sample_sales,
    connection=local_conn,
    format="delta",
    path=delta_path,
    mode="overwrite"
)

print("✅ Delta table created!")
print(f"\nLocation: {local_conn.get_path(delta_path)}")
print("\nDelta table structure:")
print("  sales.delta/")
print("  ├── _delta_log/        (transaction log)")
print("  │   └── 00000000000000000000.json")
print("  └── *.parquet          (data files)")

In [ ]:
# Read Delta table
df_read = engine.read(
    connection=local_conn,
    format="delta",
    path=delta_path
)

print("✅ Delta table read successfully!")
print(f"\nRows: {len(df_read)}")
print(df_read)

## Part 3: Append Mode - Creating Multiple Versions

In [ ]:
# Append more sales (version 1)
new_sales = pd.DataFrame({
    "order_id": [6, 7],
    "customer": ["Frank", "Grace"],
    "product": ["Tablet", "Laptop"],
    "amount": [400, 1300],
    "year": [2024, 2024],
    "month": [3, 4]
})

engine.write(
    new_sales,
    connection=local_conn,
    format="delta",
    path=delta_path,
    mode="append"  # ← Append mode!
)

print("✅ Appended 2 more orders (version 1 created)")
print(new_sales)

In [ ]:
# Read latest version
df_latest = engine.read(local_conn, format="delta", path=delta_path)

print("✅ Latest version:")
print(f"\nRows: {len(df_latest)} (was 5, now 7)")
print(f"Total: ${df_latest['amount'].sum():,}")
print("\nLast 3 orders:")
print(df_latest.tail(3))

## Part 4: Time Travel - Read Specific Versions

In [ ]:
# Read version 0 (original data)
df_v0 = engine.read(
    connection=local_conn,
    format="delta",
    path=delta_path,
    options={"versionAsOf": 0}  # ← Time travel!
)

print("✅ Time travel to version 0:")
print(f"\nRows: {len(df_v0)} (original 5 orders)")
print(f"Total: ${df_v0['amount'].sum():,}")
print("\nData:")
print(df_v0)

In [ ]:
# Compare versions
print("Version Comparison:")
print("="*50)
print(f"Version 0: {len(df_v0)} rows, ${df_v0['amount'].sum():,}")
print(f"Version 1: {len(df_latest)} rows, ${df_latest['amount'].sum():,}")
print(f"\nDifference: +{len(df_latest) - len(df_v0)} rows, +${df_latest['amount'].sum() - df_v0['amount'].sum():,}")

## Part 5: History Tracking

In [ ]:
# Get table history
history = engine.get_delta_history(
    connection=local_conn,
    path=delta_path
)

print("✅ Delta Table History:")
print("="*50)

for entry in history:
    print(f"\nVersion {entry['version']}:")
    print(f"  Operation: {entry['operation']}")
    print(f"  Timestamp: {entry['timestamp']}")
    if 'operationMetrics' in entry:
        metrics = entry['operationMetrics']
        if 'numOutputRows' in metrics:
            print(f"  Rows: {metrics['numOutputRows']}")

## Part 6: Creating More Versions for Testing

In [ ]:
# Overwrite with "bad" data (version 2)
bad_data = pd.DataFrame({
    "order_id": [999],
    "customer": ["CORRUPTED"],
    "product": ["ERROR"],
    "amount": [0],
    "year": [9999],
    "month": [99]
})

engine.write(
    bad_data,
    connection=local_conn,
    format="delta",
    path=delta_path,
    mode="overwrite"  # Oops! Overwrote everything
)

print("❌ Oops! Accidentally overwrote the table with bad data")
print(bad_data)

In [ ]:
# Check current state
df_corrupted = engine.read(local_conn, format="delta", path=delta_path)

print("Current table state (version 2):")
print(df_corrupted)
print("\n❌ Data is corrupted! Only 1 row with error values")

## Part 7: Restore to Previous Version

In [ ]:
# Restore to version 1 (before corruption)
engine.restore_delta(
    connection=local_conn,
    path=delta_path,
    version=1  # ← Restore to version 1!
)

print("✅ Restored to version 1!")
print("\nRestoring table to before corruption...")

In [ ]:
# Verify restoration
df_restored = engine.read(local_conn, format="delta", path=delta_path)

print("✅ Table restored successfully!")
print(f"\nRows: {len(df_restored)} (back to 7 orders)")
print(f"Total: ${df_restored['amount'].sum():,}")
print("\nRestored data:")
print(df_restored)

In [ ]:
# Check updated history
history_after_restore = engine.get_delta_history(local_conn, delta_path, limit=5)

print("Updated History (last 5 versions):")
print("="*50)

for entry in history_after_restore:
    version = entry['version']
    operation = entry['operation']
    print(f"Version {version}: {operation}")

## Part 8: VACUUM - Clean Old Files

In [ ]:
# VACUUM dry run (preview what would be deleted)
result_dry_run = engine.vacuum_delta(
    connection=local_conn,
    path=delta_path,
    retention_hours=0,  # For testing only
    dry_run=True,
    enforce_retention_duration=False  # ← Add this for testing!
)

print("VACUUM Dry Run Results:")
print("="*50)
print(f"Files that would be deleted: {result_dry_run['files_deleted']}")
print("\n⚠️  This is a preview - no files were actually deleted")
print("⚠️  Note: enforce_retention_duration=False is for testing only!")
print("     In production, use default retention_hours=168")


In [ ]:
# VACUUM for real (clean old files)
result_vacuum = engine.vacuum_delta(
    connection=local_conn,
    path=delta_path,
    retention_hours=0,  # For testing only
    dry_run=False,
    enforce_retention_duration=False  # ← Add this for testing!
)

print("✅ VACUUM completed!")
print("="*50)
print(f"Files deleted: {result_vacuum['files_deleted']}")
print("\n💡 In production, use:")
print("   retention_hours=168 (7 days)")
print("   enforce_retention_duration=True (default)")

## Part 9: Partitioning (with Warnings)

In [ ]:
# Create larger dataset for partitioning
import warnings

large_sales = pd.DataFrame({
    "order_id": range(1, 101),
    "customer": [f"Customer_{i}" for i in range(1, 101)],
    "product": ["Laptop", "Phone", "Tablet"] * 33 + ["Laptop"],
    "amount": [1000 + i*10 for i in range(100)],
    "year": [2024] * 100,
    "month": [i % 12 + 1 for i in range(100)]
})

print(f"Created large dataset: {len(large_sales)} rows")
print("\nSample:")
print(large_sales.head())

In [ ]:
# Write with partitioning (will emit warning)
warnings.simplefilter('always')  # Show all warnings

with warnings.catch_warnings(record=True) as w:
    warnings.simplefilter("always")
    
    engine.write(
        large_sales,
        connection=local_conn,
        format="delta",
        path="partitioned_sales.delta",
        mode="overwrite",
        options={"partition_by": ["month"]}  # ← Partition by month
    )
    
    # Show warning
    if w:
        print("⚠️  Warning emitted:")
        print(f"   {w[0].message}")

print("\n✅ Partitioned Delta table created!")
print("\nPartitioned structure:")
print("  partitioned_sales.delta/")
print("  ├── month=1/")
print("  │   └── *.parquet")
print("  ├── month=2/")
print("  │   └── *.parquet")
print("  └── ...")

## Part 10: Delta + ADLS Integration

**Note:** This section requires Azure credentials. Update with your values.

In [ ]:
# ADLS connection setup (update with your values)
from odibi.connections.azure_adls import AzureADLS

# Option 1: Direct key (for local testing)
adls_conn = AzureADLS(
    account="YOUR_STORAGE_ACCOUNT",
    container="YOUR_CONTAINER",
    path_prefix="delta_test",
    auth_mode="key_vault",
    key_vault_name="YOUR_KEY_VAULT_NAME",
    secret_name="YOUR_ACCOUNT_KEY",
    validate=False  # Set True when ready to test
)

print("⚠️  ADLS connection configured (validation disabled)")
print("\nTo test with real ADLS:")
print("1. Update account, container, and account_key")
print("2. Set validate=True")
print("3. Run the following cell")

In [ ]:
# Write Delta to ADLS (uncomment when ready)

engine.write(
    sample_sales,
    connection=adls_conn,
    format="delta",
    path="sales.delta",
    mode="overwrite"
)
print("✅ Delta table written to ADLS!")
print(f"Location: abfss://{adls_conn.container}@{adls_conn.account}.dfs.core.windows.net/{adls_conn.path_prefix}/sales.delta")

print("Uncomment code above to test with real ADLS connection")

In [ ]:
# Read Delta from ADLS (uncomment when ready)

df_from_adls = engine.read(
    connection=adls_conn,
    format="delta",
    path="sales.delta"
)
print("✅ Delta table read from ADLS!")
print(df_from_adls)

print("Uncomment code above to test with real ADLS connection")

## Part 11: SparkEngine Delta Support (Optional)

In [ ]:
# Test if Spark is available
try:
    from odibi.engine.spark_engine import SparkEngine
    SPARK_AVAILABLE = True
    print("✅ Spark is available!")
except ImportError as e:
    SPARK_AVAILABLE = False
    print("⚠️  Spark not available")
    print("   Install with: pip install -e '.[spark]'")
    print(f"   Error: {e}")

In [ ]:
if SPARK_AVAILABLE:
    # Initialize SparkEngine
    spark_engine = SparkEngine(connections={"local": local_conn})
    
    print("✅ SparkEngine initialized")
    print(f"   Spark version: {spark_engine.spark.version}")
    print(f"   Delta configured: {hasattr(spark_engine.spark, '_jsparkSession')}")
else:
    print("Skipping Spark tests (not installed)")

In [ ]:
if SPARK_AVAILABLE:
    # Write Delta with Spark
    
    spark_df = spark_engine.spark.createDataFrame(sample_sales)
    
    spark_engine.write(
        spark_df,
        connection=local_conn,
        format="delta",
        path="spark_sales.delta",
        mode="overwrite"
    )
    
    print("✅ Delta table written with Spark!")
    print(f"   Location: {local_conn.get_path('spark_sales.delta')}")
else:
    print("Skipping (Spark not available)")

In [ ]:
if SPARK_AVAILABLE:
    # Read Delta with Spark
    spark_df_read = spark_engine.read(
        connection=local_conn,
        format="delta",
        path="spark_sales.delta"
    )
    
    print("✅ Delta table read with Spark!")
    print("\nSchema:")
    spark_df_read.printSchema()
    
    print("\nData (first 5 rows):")
    spark_df_read.show(5)
else:
    print("Skipping (Spark not available)")

## Part 12: Summary and Key Takeaways

In [ ]:
print("Phase 2B: Delta Lake Support - Summary")
print("="*50)
print("\n✅ What You Learned:")
print("\n1. Delta Lake Basics:")
print("   - Write with format='delta'")
print("   - Read Delta tables")
print("   - Append mode creates versions")
print("\n2. Time Travel:")
print("   - Read specific versions with versionAsOf")
print("   - Compare different versions")
print("\n3. History & Restore:")
print("   - Track all versions with get_delta_history()")
print("   - Restore to previous version with restore_delta()")
print("\n4. VACUUM:")
print("   - Clean old files to save storage")
print("   - Dry run mode for preview")
print("\n5. Partitioning:")
print("   - Partition by low-cardinality columns")
print("   - Performance warnings")
print("\n6. ADLS Integration:")
print("   - Delta works with Azure storage")
print("   - Same API for local and cloud")
print("\n7. Spark Support:")
print("   - SparkEngine supports Delta")
print("   - Auto-configuration with delta-spark")

In [ ]:
print("\n💡 Best Practices:")
print("="*50)
print("\n1. Use Delta for production data")
print("   - ACID transactions")
print("   - Schema evolution")
print("   - Time travel")
print("\n2. Run VACUUM regularly")
print("   - Default: retention_hours=168 (7 days)")
print("   - Saves storage costs")
print("   - Keeps time travel history")
print("\n3. Partition wisely")
print("   - Low-cardinality columns only")
print("   - Each partition > 1000 rows")
print("   - Total partitions < 1000")
print("\n4. Use time travel for:")
print("   - Debugging (what changed?)")
print("   - Auditing (who wrote what?)")
print("   - Recovery (restore good version)")

In [ ]:
print("\n🎯 Next Steps:")
print("="*50)
print("\n1. Test with your ADLS connection:")
print("   - Update Part 10 with real credentials")
print("   - Write and read Delta from cloud")
print("\n2. Build a real pipeline:")
print("   - See examples/template_full.yaml")
print("   - Use format='delta' for write nodes")
print("\n3. Explore Phase 2C (Performance):")
print("   - Parallel Key Vault fetching")
print("   - Enhanced error handling")
print("   - Production setup tools")
print("\n4. Check documentation:")
print("   - docs/PHASE2_DESIGN_DECISIONS.md")
print("   - docs/SUPPORTED_FORMATS.md")
print("   - CHANGELOG.md")

## Cleanup (Optional)

In [ ]:
# Uncomment to clean up test data

import shutil
shutil.rmtree("./delta_test_data", ignore_errors=True)
print("✅ Test data cleaned up")

print("Uncomment code above to remove test data")